# Gallery of Large Galaxies

The purpose of this notebook is to build the gallery of large galaxies for the various Legacy Survey data releases.

The parent sample is a diameter-limited (*D25>5* arcsec) sample defined and documented as part of the [Legacy Survey Large Galaxy Atlas](https://github.com/moustakas/LSLGA).  

### Imports and paths

In [3]:
import os
import subprocess
import numpy as np

import fitsio
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from PIL import Image, ImageDraw, ImageFont

from astrometry.util.util import Tan
from astrometry.util.fits import fits_table, merge_tables
from legacypipe.survey import ccds_touching_wcs, LegacySurveyData

In [4]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [5]:
plt.style.use('seaborn-talk')
%matplotlib inline

### Preliminaries

Define the data release and specify the D(25) angular diameter range (in arcmin) of the galaxies in the gallery.

In [6]:
d25min, d25max = 4.0, 5.0 # [arcmin]
dr = 'dr5'
if dr == 'dr5':
    PIXSCALE = 0.262
else:
    raise NotImplementedError

DIAMFACTOR = 4

In [7]:
gallerydir = os.path.join( os.getenv('SCRATCH'), dr, 'gallery' )
galleryfile = os.path.join(gallerydir, 'gallery-large-galaxies-{}.fits'.format(dr))

In [8]:
survey = LegacySurveyData()

### Define the parent sample based on cuts to D(25).

In [7]:
def _catalog_template(nobj=1):
    cols = [
        ('GALAXY', 'S28'), 
        ('PGC', 'S10'), 
        ('RA', 'f8'), 
        ('DEC', 'f8'),
        ('TYPE', 'S8'),
        ('MULTIPLE', 'S1'),
        ('RADIUS', 'f4'),
        ('BA', 'f4'),
        ('PA', 'f4'),
        ('BMAG', 'f4'),
        ('IMAG', 'f4'),
        ('VHELIO', 'f4'),
        ('BRICKNAME', 'S17', (4,))
        ]
    catalog = Table(np.zeros(nobj, dtype=cols))
    catalog['RADIUS'].unit = 'arcsec'
    catalog['VHELIO'].unit = 'km/s'

    return catalog

In [8]:
def read_leda(gallerydir='.', d25min=0.0, d25max=1000.0, decmin=-90.0, 
              decmax=+90.0, ramin=0.0, ramax=360.0):
    """Read the parent LEDA catalog and put it in a standard format.
    
    """
    ledafile = os.path.join(gallerydir, 'leda-logd25-0.05.fits')
    print('Reading {}'.format(ledafile))
    cat = Table.read(ledafile)

    outcat = _catalog_template(len(cat))
    outcat['GALAXY'] = cat['GALAXY']
    outcat['PGC'] = cat['PGC']
    outcat['RA'] = cat['RA']
    outcat['DEC'] = cat['DEC']
    outcat['TYPE'] = cat['TYPE']
    outcat['MULTIPLE'] = cat['MULTIPLE']
    outcat['RADIUS'] = cat['D25']/2.0 # semi-major axis radius [arcsec]
    outcat['BA'] = cat['BA']
    outcat['PA'] = cat['PA']
    outcat['BMAG'] = cat['BMAG']
    outcat['IMAG'] = cat['IMAG']
    outcat['VHELIO'] = cat['VHELIO']

    these = np.where((outcat['RADIUS']*2/60.0 <= d25max) *
                     (outcat['RADIUS']*2/60.0 >= d25min) *
                     (outcat['DEC'] <= decmax) *
                     (outcat['DEC'] >= decmin) *
                     (outcat['RA'] <= ramax) *
                     (outcat['RA'] >= ramin)
                     )[0]
    outcat = outcat[these] 
    print('Found {}/{} galaxies with D25 = {}-{} arcmin.'.format(
            len(outcat), len(cat), d25min, d25max))

    return outcat

In [9]:
bricks = survey.get_bricks()
allccds = survey.get_annotated_ccds()
cut = survey.photometric_ccds(allccds)
if cut is not None:
    allccds.cut(cut)
cut = survey.ccd_cuts(allccds)
allccds.cut(cut == 0)
print('Read {} CCDs.'.format(len(allccds)))

Converted brickname from |S8 to <U8
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-run19.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S51 to <U51
Converted plver from |S6 to <U6
Got 73860 CCDs
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-decals.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S61 to <U61
Converted plver from |S6 to <U6
Got 493440

In [10]:
parent = read_leda(gallerydir=gallerydir, d25min=d25min, d25max=d25max)

Reading /global/cscratch1/sd/ioannis/dr5/gallery/leda-logd25-0.05.fits
Found 186/2143628 galaxies with D25 = 4.0-5.0 arcmin.


### Build the parent sample of galaxies in the DRX footprint

In [17]:
def _uniqccds(ccds):
    '''Get the unique set of CCD files.'''
    ccdfile = []
    [ccdfile.append('{}-{}'.format(expnum, ccdname)) for expnum,
     ccdname in zip(ccds.expnum, ccds.ccdname)]
    _, indx = np.unique(ccdfile, return_index=True)
    return ccds[indx]

In [33]:
def _galwcs(gal, factor=DIAMFACTOR):
    '''Build a simple WCS object for a single galaxy.
    
    '''
    diam = factor*np.ceil(2.0*gal['RADIUS']/PIXSCALE).astype('int16') # [pixels]
    galwcs = Tan(gal['RA'], gal['DEC'], diam/2+0.5, diam/2+0.5,
                 -PIXSCALE/3600.0, 0.0, 0.0, PIXSCALE/3600.0, 
                 float(diam), float(diam))
    return galwcs

In [12]:
def _build_sample_onegalaxy(args):
    """Filler function for the multiprocessing."""
    return build_sample_onegalaxy(*args)

In [20]:
def build_sample_onegalaxy(gal, allccds, ccdsdir, bricks, survey):
    """Wrapper function to find overlapping CCDs for a given galaxy.

    First generously find the nearest set of CCDs that are near the galaxy and
    then demand that there's 3-band coverage in a much smaller region centered
    on the galaxy.

    """
    #print('Working on {}...'.format(gal['GALAXY'].strip()))
    galwcs = _galwcs(gal)
    these = ccds_touching_wcs(galwcs, allccds)

    if len(these) > 0:
        ccds1 = _uniqccds( allccds[these] )

        # Is there 3-band coverage?
        galwcs_small = _galwcs(gal, factor=0.5)
        these_small = ccds_touching_wcs(galwcs_small, ccds1)
        ccds1_small = _uniqccds( ccds1[these_small] )

        if 'g' in ccds1_small.filter and 'r' in ccds1_small.filter and 'z' in ccds1_small.filter:
            print('For {} found {} CCDs, RA = {:.5f}, Dec = {:.5f}, Radius={:.4f} arcsec'.format(
                gal['GALAXY'].strip(), len(ccds1), gal['RA'], gal['DEC'], gal['RADIUS']))

            ccdsfile = os.path.join(ccdsdir, '{}-ccds.fits'.format(gal['GALAXY'].strip().lower()))
            #print('  Writing {}'.format(ccdsfile))
            #if os.path.isfile(ccdsfile):
            #    os.remove(ccdsfile)
            #ccds1.writeto(ccdsfile)

            # Also get the set of bricks touching this footprint.
            rad = 2*gal['RADIUS']/3600 # [degree]
            brickindx = survey.bricks_touching_radec_box(bricks,
                                                         gal['RA']-rad, gal['RA']+rad,
                                                         gal['DEC']-rad, gal['DEC']+rad)
            if len(brickindx) == 0 or len(brickindx) > 4:
                print('This should not happen!')
                pdb.set_trace()
            gal['BRICKNAME'][:len(brickindx)] = bricks.brickname[brickindx]

            return [gal, ccds1]

    return None

In [24]:
ccdsdir = os.path.join(gallerydir, 'ccds')

sampleargs = list()
for cc in parent:
    sampleargs.append( (cc, allccds, ccdsdir, bricks, survey) )

nproc = 1
if nproc > 1:
    p = multiprocessing.Pool(nproc)
    result = p.map(_build_sample_onegalaxy, sampleargs)
    p.close()
else:
    result = list()
    for args in sampleargs:
        result.append(_build_sample_onegalaxy(args))

# Remove non-matching galaxies and write out the sample
result = list(filter(None, result))
result = list(zip(*result))

outcat = vstack(result[0])
outccds = merge_tables(result[1])

if os.path.isfile(galleryfile):
    os.remove(galleryfile)
            
print('Writing {}'.format(galleryfile))
outcat.write(galleryfile)

For PGC003853 found 34 CCDs, RA = 16.27035, Dec = -6.21246, Radius=122.2141 arcsec
For NGC0520 found 35 CCDs, RA = 21.14445, Dec = 3.79420, Radius=122.2141 arcsec
For NGC0720 found 69 CCDs, RA = 28.25220, Dec = -13.73859, Radius=134.0051 arcsec
For NGC0936 found 92 CCDs, RA = 36.90585, Dec = -1.15595, Radius=134.0051 arcsec
For NGC0988 found 41 CCDs, RA = 38.86545, Dec = -9.35595, Radius=130.9547 arcsec
For UGC02302 found 45 CCDs, RA = 42.28635, Dec = 2.12703, Radius=143.5890 arcsec
For UGC03974 found 17 CCDs, RA = 115.48080, Dec = 16.80250, Radius=130.9547 arcsec
For NGC2775 found 23 CCDs, RA = 137.58375, Dec = 7.03792, Radius=127.9739 arcsec
For NGC3003 found 11 CCDs, RA = 147.14880, Dec = 33.42136, Radius=143.5890 arcsec
For NGC3044 found 69 CCDs, RA = 148.42035, Dec = 1.57959, Radius=125.0608 arcsec
For UGC05364 found 19 CCDs, RA = 149.86005, Dec = 30.74639, Radius=137.1264 arcsec
For UGC05373 found 30 CCDs, RA = 150.00015, Dec = 5.33224, Radius=146.9337 arcsec
For SDSSJ100819.08+1

### Retrieve FITS cutouts of each galaxy in each band

In [10]:
fitsdir = os.path.join(gallerydir, 'fits')
if not os.path.isdir(fitsdir):
    os.mkdir(fitsdir)

In [11]:
sample = Table.read(galleryfile)
sample

GALAXY,PGC,RA,DEC,TYPE,MULTIPLE,RADIUS,BA,PA,BMAG,IMAG,VHELIO,BRICKNAME [4]
str28,str10,float64,float64,str4,str1,float32,float32,float32,float32,float32,float32,str20
PGC003853,PGC0003853,16.27035,-6.21246,SABc,,122.214,0.74131,45.0,12.43,12.51,1095.0,0162m062 ..
NGC0520,PGC0005193,21.14445,3.7942,Sa,,122.214,0.389045,130.0,12.21,10.5,2161.0,0211p037 ..
NGC0720,PGC0006983,28.2522,-13.73859,E,,134.005,0.524807,141.9,11.15,9.31,1717.0,0281m137 ..
NGC0936,PGC0009359,36.90585,-1.15595,S0-a,,134.005,0.724436,133.0,11.2,9.4,1432.0,0368m012 ..
NGC0988,PGC0009843,38.86545,-9.35595,Sc,,130.955,0.398107,119.2,10.94,7.03,1506.0,0388m095 ..
UGC02302,PGC0010670,42.28635,2.12703,Sm,,143.589,0.954993,-999.0,14.08,-999.0,1104.0,0421p020 .. 0423p022
UGC03974,PGC0021600,115.4808,16.8025,IB,,130.955,0.977237,-999.0,13.74,14.09,272.0,1154p167 ..
NGC2775,PGC0025861,137.58375,7.03792,Sab,,127.974,0.794328,159.0,11.14,-999.0,1354.0,1375p070 ..
NGC3003,PGC0028186,147.1488,33.42136,Sbc,M,143.589,0.223872,78.8,12.25,11.08,1480.0,1470p332 .. 1472p335


In [69]:
for gal in sample[:20]:
    galaxy = gal['GALAXY'].strip().lower()

    # SIZE here should be consistent with DIAM in args.runbrick, below
    size = DIAMFACTOR*np.ceil(gal['RADIUS']/PIXSCALE).astype('int16') # [pixels]

    # Get a FITS cutout and then split the file into three individual bands.
    baseurl = 'http://legacysurvey.org/viewer-dev/fits-cutout/'
    fitsurl = '{}?ra={:.6f}&dec={:.6f}&pixscale={:.3f}&size={:g}&layer=decals-{}'.format(
        baseurl, gal['RA'], gal['DEC'], PIXSCALE, size, dr)
    fitsfile = os.path.join(fitsdir, '{}.fits'.format(galaxy))
    cmd = 'wget --continue -O {:s} "{:s}"' .format(fitsfile, fitsurl)
    print(cmd)
    os.system(cmd)

    img, hdr = fitsio.read(fitsfile, ext=0, header=True)
    for ii, band in enumerate(['g', 'r', 'z']):
        bandfile = os.path.join(fitsdir, '{}-{}.fits'.format(galaxy, band))
        print('  Writing {}'.format(bandfile))
        fitsio.write(bandfile, img[ii, :, :], clobber=True, header=hdr)        
    
    print('  Removing {}'.format(fitsfile))
    os.remove(fitsfile)

wget --continue -O /global/cscratch1/sd/ioannis/dr5/gallery/fits/pgc003853.fits "http://legacysurvey.org/viewer-dev/fits-cutout/?ra=16.270350&dec=-6.212460&pixscale=0.262&size=1868&layer=decals-dr5"
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/pgc003853-g.fits
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/pgc003853-r.fits
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/pgc003853-z.fits
  Removing /global/cscratch1/sd/ioannis/dr5/gallery/fits/pgc003853.fits
wget --continue -O /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0520.fits "http://legacysurvey.org/viewer-dev/fits-cutout/?ra=21.144450&dec=3.794200&pixscale=0.262&size=1868&layer=decals-dr5"
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0520-g.fits
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0520-r.fits
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0520-z.fits
  Removing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0520.fits
wget --continue -O /global/cscr

### Build a color image using trilogy.py

In [12]:
pngdir = os.path.join(gallerydir, 'png')
if not os.path.isdir(pngdir):
    os.mkdir(pngdir)

In [ ]:
for gal in sample[:20]:
    galaxy = gal['GALAXY'].strip().lower()
    pngfile = os.path.join(pngdir, '{}.png'.format(galaxy))

    paramfile = os.path.join(pngdir, '{}.in'.format(galaxy))
    with open(paramfile, 'w') as param:
        param.write('B\n')
        param.write(os.path.join(fitsdir, '{}-g.fits\n'.format(galaxy)))
        param.write('\n')
        param.write('G\n')
        param.write(os.path.join(fitsdir, '{}-r.fits\n'.format(galaxy)))
        param.write('\n')        
        param.write('R\n')
        param.write(os.path.join(fitsdir, '{}-z.fits\n'.format(galaxy)))
        param.write('\n')
        param.write('indir {}\n'.format(fitsdir))
        param.write('outname {}\n'.format(pngfile))
        param.write('noiselum 0.5\n')
        param.write('satpercent 0.001\n')
        param.write('samplesize 500\n')
        param.write('stampsize 5000\n')
        #param.write('scaling {}\n'.format(os.path.join(gallerydir, 'levels.txt')))
        param.write('colorsatfac 0.9\n')
        param.write('deletetests 1\n')
        param.write('showstamps 0\n')
        param.write('show 0\n')
        param.write('legend 0\n')
        param.write('testfirst 0\n')
        
    trilogy = os.path.join(os.getenv('LEGACYPIPE_DIR'), 'py', 'legacyanalysis', 'trilogy.py')
    cmd = 'python {} {}'.format(trilogy, paramfile)
    #cmd = ('python', trilogy , paramfile)
    print(cmd)
    #print(' '.join(cmd))
    #subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    os.system(cmd)
    for txt in ('levels.txt', 'trilogyfilterlog.txt'):
        thisfile = os.path.join(os.getenv('LEGACYPIPE_DIR'), 'doc', 'nb', txt)
        os.remove(thisfile)
    os.remove(os.path.join(pngdir, '{}_filters.txt'.format(galaxy)))

In [24]:
# Annotate and add a scale bar!!

### Finally, assemble the webpage

In [ ]:
def make_html(htmlfile, pnglist=pnglist, npngcols=3, title=title, sample=sample1)

    html_path = file_dirname(htmlfile)
    png_path = html_path+'/png/'

    pngfiles = strtrim(sample1.object,2)+'.png'
    good = where(file_test(png_path+pngfiles) eq 1)
    sample = sample1[good]
    pngfiles = pngfiles[good]
    
    npngfiles = n_elements(pngfiles)
    
    if (not keyword_set(silent)) then splog, 'Found '+$
      string(npngfiles,format='(I0)')+' PNG files'

; generate the web page HTML page
    rootpngfiles = repstr(pngfiles,'.png','')
    nmenucols = npngcols

    if (not keyword_set(only_png)) then begin

       xwidth = string(fix(100/float(npngcols)),format='(I0)')
       
       if (not keyword_set(silent)) then splog, 'Writing '+htmlfile
       
       openw, lun1, htmlfile,/get_lun
       printf, lun1, '<html>'
       printf, lun1, '<head>'
       printf, lun1, '<title>'+title+'</title>'
       printf, lun1, '<style type="text/css">'
       printf, lun1, '<!--'
       printf, lun1, 'body {'
       printf, lun1, '   color: white;'
       printf, lun1, '   background-color: black;'
       printf, lun1, '   text-align: center;'
       printf, lun1, '   border: 0;'
       printf, lun1, '   font-size: 100%;'
       printf, lun1, '   font-weight: bold;'
       printf, lun1, '}'
       printf, lun1, 'td {'
       printf, lun1, '   font-size: 100%;'
       printf, lun1, '   font-weight: bold;'
       printf, lun1, '   text-align: center;'
       printf, lun1, '}'
       printf, lun1, 'a {'
       printf, lun1, '   color: cyan;'
       printf, lun1, '   visited: cyan;'
       printf, lun1, '   text-decoration: none;'
       printf, lun1, '}'
       printf, lun1, '.center {'
       printf, lun1, '   text-align: center;'
       printf, lun1, '}'
       printf, lun1, '.left {'
       printf, lun1, '   text-align: left;'
       printf, lun1, '}'
       printf, lun1, '.smaller {'
       printf, lun1, '   font-size: 90%;'
       printf, lun1, '}'
       printf, lun1, 'h1 {color: gold;}'
       printf, lun1, '</style>'
       printf, lun1, '-->'
       printf, lun1, ' '
       printf, lun1, '<link href="http://legacysurvey.org/test/assets/css/bootstrap.min.css" rel="stylesheet" type="text/css">'
       printf, lun1, '<link href="http://legacysurvey.org/test/assets/css/rst.css" rel="stylesheet" type="text/css">'
       printf, lun1, '<link href="http://legacysurvey.org/test/assets/css/code.css" rel="stylesheet" type="text/css">'
       printf, lun1, '<link href="http://legacysurvey.org/test/assets/css/colorbox.css" rel="stylesheet" type="text/css">'
       printf, lun1, '<link href="http://legacysurvey.org/test/assets/css/theme.css" rel="stylesheet" type="text/css">'
       printf, lun1, '<link href="http://legacysurvey.org/test/assets/css/custom.css" rel="stylesheet" type="text/css">'
       printf, lun1, ' '
       printf, lun1, '</head>'
       printf, lun1, '<body>'
       printf, lun1, '<h1>Image Gallery of Large Galaxies</h1>'
       printf, lun1, '<p>Color mosaics generated using <a class="reference external" href="http://www.stsci.edu/~dcoe/trilogy/Intro.html">trilogy.py</a>.  The white bar represents 30 arcsec.</p>'
       printf, lun1, '<br />'
       printf, lun1, '<table align="center" cellpadding="5" border="1" cellspacing="5" width="100%">'
       printf, lun1, '<tbody>'

       for i = 0L, ceil(npngfiles/float(npngcols))-1 do begin
; images          
          printf, lun1, '<tr>'
          for j = 0L, npngcols-1 do begin
             indx = j + i*npngcols
             if (indx le npngfiles-1L) then begin
                printf, lun1, '<td width="'+xwidth+'%"><a href="png/'+pngfiles[indx]+'">'+$
                  '<img width="100%" align="center" valign="center" src="png/thumb-'+pngfiles[indx]+'"></a></td>'
             endif
          endfor
          printf, lun1, '</tr>'

; image captions          
          printf, lun1, '<tr>'
          for j = 0L, npngcols-1L do begin
             indx = j + i*npngcols
             if (indx le npngfiles-1L) then begin
                printf, lun1, '<td width="'+xwidth+'%" class="left">'+$
                  '<a target="_blank" href="http://legacysurvey.org/viewer/?layer=decals-dr2&ra='+$
                  strtrim(string(sample[indx].ra,format='(F12.4)'),2)+'&dec='+$
                  strtrim(string(sample[indx].dec,format='(F12.5)'),2)+'&zoom=14">'+$
                  strtrim(sample[indx].object,2)+'</a>'
                printf, lun1, '</td>'
             endif
          endfor
          printf, lun1, '</tr>'
          
       endfor 

       printf, lun1, '</tbody>'
       printf, lun1, '</table>'
       printf, lun1, '</body>'
       printf, lun1, '</html>'
       free_lun, lun1

In [ ]:
sample = mrdfits(gallerydir+'gallery_sample.fits.gz',1)
gal = strcompress(sample.object,/remove)
these = where(file_test(gallerydir+'png/'+gal+'.png'),nthese)

; missing data on these       
       skip = ['IC1256','IC2327','IC534','MCG0-32-4','NGC2576',$
         'NGC2743','NGC3323','NGC4382','NGC4536','NGC5534',$
         'NGC5656','NGC5806','NGC5838','NGC5850','NGC5956',$
         'NGC7046','NGC7081','UGC10176','UGC10547','UGC10831',$
         'UGC10837','UGC10905','UGC11859','UGC4285','UGC4340',$
         'UGC6440','UGC6821','UGC7170','UGC9117','UGC9760',$
         'UGC9900','UGC9977','IC1516','MCG-1-59-27','CGCG119-119',$
         'MCG2-55-4','NGC5983','UGC10023','UGC11807','UGC4571',$
         'UGC4884','UGC6379','UGC9782','UGC9949','MCG0-29-28']
       keep = these
       match, gal[these], skip, m1, m2
       remove, m1, keep

       title = 'DECaLS/DR2 Image Gallery'
       
       htmlfile = gallerydir+'index.html'
       make_html, htmlfile, npngcols=5, title=title, $
         pnglist=pngfiles, sample=sample[keep]
    endif
